In [1]:
import pandas as pd
import numpy as np


#1.# Load datasets
users = pd.read_csv('/Users/paramanandbhat/Downloads/Non_personalisedrecommendersystemsinpython-201024-234034 (1)/user_demographics.csv')

#Reading ratings file:
ratings= pd.read_csv('/Users/paramanandbhat/Downloads/Non_personalisedrecommendersystemsinpython-201024-234034 (1)/ratings.csv')

#Reading items file:
movie_info = pd.read_csv('/Users/paramanandbhat/Downloads/Non_personalisedrecommendersystemsinpython-201024-234034 (1)/movie_info.csv')


In [2]:
### Exploring user data

In [3]:
# shape of the users data
print(users.shape)
# view the users data
users.head()

(943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
pd.isnull(users).sum()

user_id       0
age           0
sex           0
occupation    0
zip_code      0
dtype: int64

In [5]:
### Exploring ratings data

In [6]:
# shape of the data
print(ratings.shape)
# view the ratings data
ratings.head()

(100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
ratings[(ratings['user_id'] == 1)&(ratings['movie_id'] == 100)]

,user_id,movie_id,rating,unix_timestamp
17672,1,100,5,878543541


In [8]:
pd.isnull(ratings).sum()

user_id           0
movie_id          0
rating            0
unix_timestamp    0
dtype: int64

In [9]:
### Exploring Movie Information data

In [10]:
# shape of the data
print(movie_info.shape)
# view the items file
movie_info.head()

(1682, 22)


,movie id,movie title,release date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-95,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-95,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-95,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-95,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-95,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [11]:
# Check missing values in movie information
pd.isnull(movie_info).sum()

movie id        0
movie title     0
release date    1
unknown         0
Action          0
Adventure       0
Animation       0
Children's      0
Comedy          0
Crime           0
Documentary     0
Drama           0
Fantasy         0
Film-Noir       0
Horror          0
Musical         0
Mystery         0
Romance         0
Sci-Fi          0
Thriller        0
War             0
Western         0
dtype: int64

In [12]:
 ## 3.  Merging Movie information to ratings dataframe 

In [13]:
ratings = ratings.merge(movie_info[['movie id','movie title']], how='left', left_on = 'movie_id', right_on = 'movie id')

In [14]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp,movie id,movie title
0,196,242,3,881250949,242,Kolya (1996)
1,186,302,3,891717742,302,L.A. Confidential (1997)
2,22,377,1,878887116,377,Heavyweights (1994)
3,244,51,2,880606923,51,Legends of the Fall (1994)
4,166,346,1,886397596,346,Jackie Brown (1997)


In [15]:
ratings['movie'] = ratings['movie_id'].map(str) + str(': ') + ratings['movie title'].map(str)

In [16]:
ratings.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp', 'movie id',
       'movie title', 'movie'],
      dtype='object')

In [17]:
ratings = ratings.drop(['movie id', 'movie title', 'movie_id','unix_timestamp'], axis = 1)

In [18]:
ratings = ratings[['user_id','movie','rating']]

In [19]:
movie_counts = ratings['movie'].value_counts()
ratings = ratings[(ratings['movie'].isin(movie_counts[movie_counts >= 100].index))]

In [20]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie.unique().shape[0]

In [21]:
n_users, n_items

(943, 338)

In [22]:
user_movie_matrix = ratings.pivot(index = 'user_id', columns = 'movie', values = 'rating')

In [23]:
user_movie_matrix

movie,100: Fargo (1996),1012: Private Parts (1997),1016: Con Air (1997),1028: Grumpier Old Men (1995),1047: Multiplicity (1996),109: Mystery Science Theater 3000: The Movie (1996),"111: Truth About Cats & Dogs, The (1996)",116: Cold Comfort Farm (1995),"117: Rock, The (1996)",118: Twister (1996),...,"928: Craft, The (1996)",92: True Romance (1993),93: Welcome to the Dollhouse (1995),94: Home Alone (1990),95: Aladdin (1992),96: Terminator 2: Judgment Day (1991),97: Dances with Wolves (1990),"98: Silence of the Lambs, The (1991)",99: Snow White and the Seven Dwarfs (1937),9: Dead Man Walking (1995)
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,5.0,5.0,3.0,3.0,3.0,...,NaN,3.0,5.0,2.0,4.0,5.0,3.0,4.0,3.0,5.0
2,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
940,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,5.0,NaN,4.0,NaN,3.0
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
ratings = ratings.merge(users[['user_id','sex']], how = 'left', on = 'user_id')
ratings = ratings[['user_id','sex','movie','rating']]

In [25]:
## 4. Non Personalised Recommender Systems using average ratings

In [26]:
user_movie_matrix.mean(axis=0).sort_values(ascending=False).head(5)

movie
408: Close Shave, A (1995)              4.491071
318: Schindler's List (1993)            4.466443
169: Wrong Trousers, The (1993)         4.466102
483: Casablanca (1942)                  4.456790
64: Shawshank Redemption, The (1994)    4.445230
dtype: float64

In [27]:
## 5. Non Personalised Recommender Systems using number of ratings or rating count 

In [28]:
user_movie_matrix.count(axis=0).sort_values(ascending=False).head(5)

movie
50: Star Wars (1977)              583
258: Contact (1997)               509
100: Fargo (1996)                 508
181: Return of the Jedi (1983)    507
294: Liar Liar (1997)             485
dtype: int64

In [29]:
## 6. Non Personalised Recommender Systems using count of ratings 4 and above 

In [30]:
user_movie_matrix.apply(pd.value_counts)

/var/folders/c7/xx2r_j1j5jnchz6414lhvm3h0000gn/T/ipykernel_27473/3374582279.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  user_movie_matrix.apply(pd.value_counts)


movie,100: Fargo (1996),1012: Private Parts (1997),1016: Con Air (1997),1028: Grumpier Old Men (1995),1047: Multiplicity (1996),109: Mystery Science Theater 3000: The Movie (1996),"111: Truth About Cats & Dogs, The (1996)",116: Cold Comfort Farm (1995),"117: Rock, The (1996)",118: Twister (1996),...,"928: Craft, The (1996)",92: True Romance (1993),93: Welcome to the Dollhouse (1995),94: Home Alone (1990),95: Aladdin (1992),96: Terminator 2: Judgment Day (1991),97: Dances with Wolves (1990),"98: Silence of the Lambs, The (1991)",99: Snow White and the Seven Dwarfs (1937),9: Dead Man Walking (1995)
1.0,14,8,8,10,14,8,14,5,9,20,...,9,6,6,7,2,6,6,6,1,11
2.0,18,7,11,40,33,16,18,9,37,59,...,16,4,9,35,15,20,26,10,12,18
3.0,70,22,50,47,54,39,92,25,92,87,...,45,30,20,46,58,43,57,30,59,59
4.0,179,50,46,36,27,46,118,50,163,92,...,22,48,52,37,91,123,93,163,64,114
5.0,227,13,22,15,6,21,30,36,77,35,...,12,16,25,12,53,103,74,181,36,97


In [31]:
df_four = user_movie_matrix.apply(lambda x: x[x>=4]).count(axis=0) / user_movie_matrix.apply(lambda x: x).count(axis=0)
df_four.sort_values(ascending = False).head(5)

movie
479: Vertigo (1958)                     0.905028
64: Shawshank Redemption, The (1994)    0.901060
408: Close Shave, A (1995)              0.892857
169: Wrong Trousers, The (1993)         0.889831
318: Schindler's List (1993)            0.889262
dtype: float64

In [32]:
## 7. Weak Personalisation using Gender Information

In [33]:
# Ensure all columns except 'sex' are numeric
# Assuming all other columns are movie ratings which should be numeric
for col in user_movie_matrix.columns:
    if col != 'sex':
        user_movie_matrix[col] = pd.to_numeric(user_movie_matrix[col], errors='coerce')


In [35]:
# Filter for male and female users
df_m = user_movie_matrix[user_movie_matrix['sex'] == 'M']
df_f = user_movie_matrix[user_movie_matrix['sex'] == 'F']

# Calculate mean, excluding 'sex' column
df_m_mean = df_m.drop(columns='sex').mean(axis=0).sort_values(ascending=False)
df_f_mean = df_f.drop(columns='sex').mean(axis=0).sort_values(ascending=False)

In [36]:

# Compute the difference
dif_g = df_f_mean - df_m_mean
dif_g_sorted = dif_g.sort_values(ascending=False)
dif_g_sorted

movie
476: First Wives Club, The (1996)              0.748951
485: My Fair Lady (1964)                       0.635686
29: Batman Forever (1995)                      0.635452
38: Net, The (1995)                            0.625616
451: Grease (1978)                             0.612793
                                                 ...   
199: Bridge on the River Kwai, The (1957)     -0.621978
554: Waterworld (1995)                        -0.664553
156: Reservoir Dogs (1992)                    -0.693785
92: True Romance (1993)                       -0.727273
177: Good, The Bad and The Ugly, The (1966)   -0.762913
Length: 338, dtype: float64

In [37]:
df_m = df_m.drop(['sex'],axis = 1)
df_f = df_f.drop(['sex'],axis = 1)

In [38]:
df_m_four = df_m.apply(lambda x: x[x>=4]).count(axis=0).sort_values(ascending=False) / df_m.apply(lambda x: x).count(axis=0)
df_f_four = df_f.apply(lambda x: x[x>=4]).count(axis=0).sort_values(ascending=False) / df_f.apply(lambda x: x).count(axis=0)

In [39]:
dif_g = df_f_four - df_m_four
dif_g.sort_values(ascending=False)

movie
476: First Wives Club, The (1996)    0.344353
38: Net, The (1995)                  0.306934
225: 101 Dalmatians (1996)           0.298450
485: My Fair Lady (1964)             0.279403
402: Ghost (1990)                    0.274242
                                       ...   
523: Cool Hand Luke (1967)          -0.297573
331: Edge, The (1997)               -0.331281
156: Reservoir Dogs (1992)          -0.355367
92: True Romance (1993)             -0.357955
205: Patton (1970)                  -0.358178
Length: 338, dtype: float64

In [40]:
'''Clearly some new movies have popped up here. We saw how easy it was to create non personalised recommender systems for movies and we did not need to remove old movies as when it comes to movies users are not that worried about how old is a particular movie rather they are interested in actors, genre etc.'''

'Clearly some new movies have popped up here. We saw how easy it was to create non personalised recommender systems for movies and we did not need to remove old movies as when it comes to movies users are not that worried about how old is a particular movie rather they are interested in actors, genre etc.'